In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

In [3]:
df = pd.read_pickle("Datasets/pickle/embeddings_cleaned.pkl")

In [4]:
df.head(4)

,_id,subject_content_text_cleaned,minilm_embeddings_cleaned
93485,PMOPG/D/2023/0015581,pgrj,"[-0.09041333943605423, 0.013560930266976357, -..."
149941,PMOPG/D/2023/0023231,pgrj,"[-0.09041333943605423, 0.013560930266976357, -..."
137640,DEAID/E/2023/0005953,ट्रांसफर में व्याप्त भ्रष्टाचार छुपाने हेतु से...,"[0.06316458433866501, 0.014884322881698608, -0..."
26768,DOEXP/E/2023/0000087,expenditure pay upgradation anomalies in pay...,"[-0.06277257204055786, -0.006890749093145132, ..."


In [5]:
df["minilm_embeddings_len"] = df["minilm_embeddings_cleaned"].progress_apply(lambda row: len(row))

100%|██████████████████████████████| 174784/174784 [00:00<00:00, 1141300.46it/s]


In [6]:
df.minilm_embeddings_len.value_counts()

minilm_embeddings_len
384    174784
Name: count, dtype: int64

In [7]:
from sklearn.utils import shuffle

df = shuffle(df)

In [8]:
df

,_id,subject_content_text_cleaned,minilm_embeddings_cleaned,minilm_embeddings_len
171831,PMOPG/D/2023/0023139,frank,"[-0.030974000692367554, 0.032949354499578476, ...",384
31022,MOSJE/E/2023/0000087,तकनीकी शिक्षा विभाग भोपाल के उच्च कार्यालय से ...,"[0.0010111008305102587, 0.031157847493886948, ...",384
160363,PMOPG/E/2023/0025351,wwwmangaloreancom20yearoldstudentdiesafterbusr...,"[0.03162650391459465, -0.016353311017155647, 0...",384
16378,GOVAS/I/2023/0000848,due to network issuecaller voice was not audib...,"[-0.017278095707297325, -0.025179877877235413,...",384
148796,PMOPG/D/2023/0019729,pgsat,"[-0.02158086933195591, -0.023720789700746536, ...",384
...,...,...,...,...
64590,MOLBR/E/2023/0004071,labour and employment transfer related issues...,"[-0.02575930580496788, -0.03641565889120102, -...",384
174159,CBODT/E/2023/0006100,central board of direct taxes income tax dire...,"[-0.03453265503048897, -0.008983409963548183, ...",384
28861,GOVGJ/E/2023/0000354,दवारका ज़िल्लाना नायब ज़िल्ल्ला विकास अधिकारी...,"[-0.027166463434696198, 0.060572899878025055, ...",384
111610,PMOPG/E/2023/0017854,i am a widow and i am sending my written compl...,"[-0.024659985676407814, 0.04469205066561699, 0...",384


In [11]:
df_18 = df[:15750]

In [12]:
embeddings = np.array([
    np.array(em) for em in list(df_18.minilm_embeddings_cleaned)
])

In [13]:
embeddings.shape

(15750, 384)

# Clustering Modeling (Agglomerative)

In [14]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [15]:
n_clusters = list()
silhouette_scores = list()
davies_bouldin_scores = list()

## Model 1

In [16]:
model_1 = AgglomerativeClustering(
    n_clusters=4500,
    linkage="ward",
    metric="euclidean"
)

In [18]:
res_1 = model_1.fit_predict(embeddings)
df_18["aggClustering_4500"] = model_1.labels_
df_18.head(3)

/tmp/ipykernel_4915/3666578613.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_18["aggClustering_4500"] = model_1.labels_


,_id,subject_content_text_cleaned,minilm_embeddings_cleaned,minilm_embeddings_len,aggClustering_4500
171831,PMOPG/D/2023/0023139,frank,"[-0.030974000692367554, 0.032949354499578476, ...",384,2624
31022,MOSJE/E/2023/0000087,तकनीकी शिक्षा विभाग भोपाल के उच्च कार्यालय से ...,"[0.0010111008305102587, 0.031157847493886948, ...",384,3810
160363,PMOPG/E/2023/0025351,wwwmangaloreancom20yearoldstudentdiesafterbusr...,"[0.03162650391459465, -0.016353311017155647, 0...",384,3795


In [19]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

sil = silhouette_score(embeddings, model_1.labels_)
db = davies_bouldin_score(embeddings, model_1.labels_)

/home/kushalbellani18/.local/lib/python3.11/site-packages/sklearn/utils/extmath.py:208: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [20]:
n_clusters.append(4500)
silhouette_scores.append(sil)
davies_bouldin_scores.append(db)

In [21]:
sil, db

(0.2832914562101908, 1.0492001168136391)

# Model 2

In [22]:
model_2 = AgglomerativeClustering(
    n_clusters=8700,
    linkage="ward",
    metric="euclidean"
)

In [23]:
res_2 = model_2.fit_predict(embeddings)
df_18["aggClustering_8700"] = model_2.labels_

df_18.head(4)

/tmp/ipykernel_4915/2020128371.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_18["aggClustering_8700"] = model_2.labels_


,_id,subject_content_text_cleaned,minilm_embeddings_cleaned,minilm_embeddings_len,aggClustering_4500,aggClustering_8700
171831,PMOPG/D/2023/0023139,frank,"[-0.030974000692367554, 0.032949354499578476, ...",384,2624,2624
31022,MOSJE/E/2023/0000087,तकनीकी शिक्षा विभाग भोपाल के उच्च कार्यालय से ...,"[0.0010111008305102587, 0.031157847493886948, ...",384,3810,951
160363,PMOPG/E/2023/0025351,wwwmangaloreancom20yearoldstudentdiesafterbusr...,"[0.03162650391459465, -0.016353311017155647, 0...",384,3795,7591
16378,GOVAS/I/2023/0000848,due to network issuecaller voice was not audib...,"[-0.017278095707297325, -0.025179877877235413,...",384,4015,2007


In [24]:
sil = silhouette_score(embeddings, model_2.labels_)
db = davies_bouldin_score(embeddings, model_2.labels_)

n_clusters.append(8700)
silhouette_scores.append(sil)
davies_bouldin_scores.append(db)

/home/kushalbellani18/.local/lib/python3.11/site-packages/sklearn/utils/extmath.py:208: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [25]:
sil, db

(0.29008489838595214, 0.46175818616413294)

# Model 3

In [ ]:
model_3 = AgglomerativeClustering(
    n_clusters=6900,
    linkage="ward",
    metric="euclidean"
)

In [ ]:
res_3 = model_3.fit_predict(embeddings)
df_18["aggClustering_6900"] = model_3.labels_

df_18.head(4)

In [ ]:
sil = silhouette_score(embeddings, model_3.labels_)
db = davies_bouldin_score(embeddings, model_3.labels_)

n_clusters.append(720)
silhouette_scores.append(sil)
davies_bouldin_scores.append(db)

In [ ]:
sil, db

In [26]:
sources_df = pd.DataFrame({"n_clusters": n_clusters, "silhouette scores": silhouette_scores, "davies_bouldin_scores": davies_bouldin_scores})

In [27]:
sources_df

,n_clusters,silhouette scores,davies_bouldin_scores
0,4500,0.283291,1.049200
1,8700,0.290085,0.461758


# Add to google sheets

In [28]:
import pygsheets
gc = pygsheets.authorize(service_file="API/gSheet.json")
url = "https://docs.google.com/spreadsheets/d/16xf0r0qA2UJGa348M8KzfGRdYBtcg1LykduW_WrC0RE/edit#gid=0"

In [30]:
sh = gc.open_by_url(url)

wks = sh.worksheet_by_title("Res_15k_part_2")
wks.set_dataframe(df_18[["_id", "subject_content_text_cleaned", "aggClustering_4500", "aggClustering_8700"]], (1, 1))

/home/kushalbellani18/.local/lib/python3.11/site-packages/pygsheets/worksheet.py:1430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('unicode').replace('<NA>', nan)
/home/kushalbellani18/.local/lib/python3.11/site-packages/pygsheets/worksheet.py:1430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('unicode').replace('<NA>', nan)
